In [18]:
import numpy as np
import collections
import matplotlib.pyplot as plt

In [103]:
t_slot = 500 / 1000

###
### Below we have implemented the probabilistic model for collisions using a reverse-exponential distribution
### 

possible_n_transmissions = [1] + list(range(5, 55, 5)) # This value represents the number of simultaneous transmissions, similar to number of nodes

# num_slots = 11
alpha = 0.3

for num_slots in [5, 11, 18]:
    print(f"Num slots:{num_slots}")
    t_max = num_slots * t_slot

    # Select samples from an exponential distribution
    mean = t_max * alpha
    
    for n_transmissions in possible_n_transmissions:
        num_collisions = [0 for _ in range(num_slots)]
        slot_counts = [0 for _ in range(num_slots)]
        num_colliding_nodes = 0
        num_colliding_nodes_elastic = 0
        
        n_iters = 500
        for _ in range(n_iters):
            samples = np.random.exponential(mean, n_transmissions)
            # print(samples)
            # print("Max before clamping", max(samples))

            # Since samples can be out of bounds i.e. greater than the last slot we need to clamp it
            samples = np.clip(samples, 0, t_max)
            # print("Max after clamping", max(samples))

            # plt.hist(samples, bins=num_slots)
            # plt.show()

            # We want a reverse-exponential distribution so that successful transmissions are first
            samples = t_max - samples
            # print(samples)

            # Clamp samples to the nearest bottom slot
            samples = t_slot * np.floor(samples / t_slot)
            # print(samples)
            # print(set(samples))

            # Pick slot number
            slots = samples / t_slot # slots transmissions picked
            # print(set(slots))
            
            slot_collisions = [0 for _ in range(num_slots)]
            # Figure out which slots had collisions
            for slot in slots:
                slot = int(slot)
                slot_collisions[slot] = slot_collisions[slot] + 1
                slot_counts[slot] = slot_counts[slot] + 1
            
            for index, slot in enumerate(slot_collisions):
                if slot > 1:
                    num_collisions[index] = num_collisions[index] + 1
                    num_colliding_nodes += slot
                    if index < 9:
                        num_colliding_nodes_elastic += slot

        num_collisions = np.array(num_collisions) / n_iters
        slot_counts = np.array(slot_counts)
        
        p_collision = num_colliding_nodes / (n_iters * n_transmissions)
        p_collision_elastic = num_colliding_nodes_elastic / (n_iters * n_transmissions)

        print(f"n_trans={n_transmissions}", p_collision, p_collision_elastic, num_collisions, slot_counts)
        
    print("")

Num slots:5
n_trans=1 0.0 0.0 [0. 0. 0. 0. 0.] [ 45  24  57 121 253]
n_trans=5 0.7244 0.7244 [0.054 0.044 0.138 0.342 0.812] [ 162  166  326  613 1233]
n_trans=10 0.8674 0.8674 [0.142 0.154 0.368 0.744 0.978] [ 329  347  661 1268 2395]
n_trans=15 0.9270666666666667 0.9270666666666667 [0.268 0.256 0.594 0.94  1.   ] [ 501  483  985 1884 3647]
n_trans=20 0.9514 0.9514 [0.39  0.362 0.73  0.976 1.   ] [ 668  619 1317 2495 4901]
n_trans=25 0.97112 0.97112 [0.54  0.488 0.864 0.992 1.   ] [ 876  787 1645 3107 6085]
n_trans=30 0.9806666666666667 0.9806666666666667 [0.64  0.572 0.924 0.994 1.   ] [1073  928 1935 3734 7330]
n_trans=35 0.9859428571428571 0.9859428571428571 [0.696 0.676 0.962 1.    1.   ] [1195 1153 2253 4292 8607]
n_trans=40 0.99065 0.99065 [0.808 0.736 0.966 1.    1.   ] [1436 1310 2534 4938 9782]
n_trans=45 0.9935555555555555 0.9935555555555555 [0.818 0.814 0.978 1.    1.   ] [ 1568  1540  2857  5586 10949]
n_trans=50 0.9956 0.9956 [0.87  0.846 0.99  1.    1.   ] [ 1726  1613  